In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import statistics
from statistics import stdev

## Part 3 section 0 - Load Part 1 CSV file

In [2]:
df_csv = pd.read_csv('C:/Users/Mayank Solanki/Downloads/mayank-test/pr.data.0.Current', delimiter='\t')

#### cleaning of whitespaces

In [3]:
df_csv.rename(columns = {'series_id        ':'series_id', '       value':'value'}, inplace = True)

In [4]:
df_csv

,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.600,NaN
1,PRS30006011,1995,Q02,2.100,NaN
2,PRS30006011,1995,Q03,0.900,NaN
3,PRS30006011,1995,Q04,0.100,NaN
4,PRS30006011,1995,Q05,1.400,NaN
...,...,...,...,...,...
33814,PRS88003203,2021,Q03,116.951,R
33815,PRS88003203,2021,Q04,120.007,R
33816,PRS88003203,2021,Q05,116.110,R
33817,PRS88003203,2022,Q01,123.905,R


## Part 3 section 0 - Load Part 2 JSON file

In [5]:
with open('C:/Users/Mayank Solanki/Downloads/assessment-scripts/datausa.json') as file:
    data = json.load(file)

In [6]:
df_json = pd.json_normalize(data, record_path=['data'])

In [7]:
df_json.rename(columns = {'ID Year':'year'}, inplace = True)

In [8]:
df_json

,ID Nation,Nation,year,Year,Population,Slug Nation
0,01000US,United States,2020,2020,326569308,united-states
1,01000US,United States,2019,2019,324697795,united-states
2,01000US,United States,2018,2018,322903030,united-states
3,01000US,United States,2017,2017,321004407,united-states
4,01000US,United States,2016,2016,318558162,united-states
5,01000US,United States,2015,2015,316515021,united-states
6,01000US,United States,2014,2014,314107084,united-states
7,01000US,United States,2013,2013,311536594,united-states


#### filter for year [2013, 2018] inclusive

In [9]:
mask = (df_json['year'] >= 2013) & (df_json['year'] <= 2018)

### Part 3 section 1 - Calculate Mean()

In [10]:
population_mean = df_json.loc[mask, 'Population'].mean()

In [11]:
population_mean

317437383.0

## Part 3 section 1 - Calculate Standard Deviation

In [12]:
population_stdev = df_json.loc[mask, 'Population'].std()

In [13]:
population_stdev

4257089.5415293295

## Part 3 section 2 - to find best year for each series_id

#### group by series_id & year to new dataframe

In [14]:
df_csv_sorted = df_csv.sort_values(by=['series_id', 'year'], ascending=True)

#### sum of values for each year

In [15]:
df2=df_csv_sorted.groupby(['series_id', 'year'], as_index=False)['value'].sum()

In [16]:
df2

,series_id,year,value
0,PRS30006011,1995,7.100
1,PRS30006011,1996,-0.500
2,PRS30006011,1997,4.400
3,PRS30006011,1998,4.200
4,PRS30006011,1999,-7.700
...,...,...,...
7801,PRS88003203,2018,551.596
7802,PRS88003203,2019,564.447
7803,PRS88003203,2020,567.794
7804,PRS88003203,2021,580.551


#### finding the max value for each series_id hence giving output of the best year

In [17]:
df2=df2.loc[df2.groupby('series_id')['value'].idxmax()]

In [18]:
df2

,series_id,year,value
27,PRS30006011,2022,13.200
54,PRS30006012,2021,13.100
59,PRS30006013,1998,733.348
99,PRS30006021,2010,17.700
127,PRS30006022,2010,12.500
...,...,...,...
7673,PRS88003192,2002,285.200
7720,PRS88003193,2021,647.437
7734,PRS88003201,2007,21.700
7776,PRS88003202,2021,24.000


#### reset index values

In [19]:
df2 = df2.reset_index(drop=True)

In [20]:
df2

,series_id,year,value
0,PRS30006011,2022,13.200
1,PRS30006012,2021,13.100
2,PRS30006013,1998,733.348
3,PRS30006021,2010,17.700
4,PRS30006022,2010,12.500
...,...,...,...
277,PRS88003192,2002,285.200
278,PRS88003193,2021,647.437
279,PRS88003201,2007,21.700
280,PRS88003202,2021,24.000


#### exported the result to csv file

In [22]:
df2.to_csv('C:/Users/Mayank Solanki/Downloads/assessment-scripts/part-3-analysis.csv')

## Part 3 section 3

#### merging Dataframes Part 1 & Part 2 on year column

In [23]:
df3 = df_csv.merge(df_json,on='year')

In [24]:
df3

,series_id,year,period,value,footnote_codes,ID Nation,Nation,Year,Population,Slug Nation
0,PRS30006011,2013,Q01,1.300,NaN,01000US,United States,2013,311536594,united-states
1,PRS30006011,2013,Q02,0.200,NaN,01000US,United States,2013,311536594,united-states
2,PRS30006011,2013,Q03,0.200,NaN,01000US,United States,2013,311536594,united-states
3,PRS30006011,2013,Q04,0.700,NaN,01000US,United States,2013,311536594,united-states
4,PRS30006011,2013,Q05,0.600,NaN,01000US,United States,2013,311536594,united-states
...,...,...,...,...,...,...,...,...,...,...
9835,PRS88003203,2020,Q01,114.161,R,01000US,United States,2020,326569308,united-states
9836,PRS88003203,2020,Q02,114.821,R,01000US,United States,2020,326569308,united-states
9837,PRS88003203,2020,Q03,111.252,R,01000US,United States,2020,326569308,united-states
9838,PRS88003203,2020,Q04,114.007,R,01000US,United States,2020,326569308,united-states


#### Data cleaning - trimming of whitespaces from series_id column

In [25]:
df3['series_id'] = df3['series_id'].str.strip()

#### drop unwanted columns

In [26]:
df3.drop(['footnote_codes', 'ID Nation', 'Nation', 'Year', 'Slug Nation'], axis=1, inplace=True)

In [27]:
df3

,series_id,year,period,value,Population
0,PRS30006011,2013,Q01,1.300,311536594
1,PRS30006011,2013,Q02,0.200,311536594
2,PRS30006011,2013,Q03,0.200,311536594
3,PRS30006011,2013,Q04,0.700,311536594
4,PRS30006011,2013,Q05,0.600,311536594
...,...,...,...,...,...
9835,PRS88003203,2020,Q01,114.161,326569308
9836,PRS88003203,2020,Q02,114.821,326569308
9837,PRS88003203,2020,Q03,111.252,326569308
9838,PRS88003203,2020,Q04,114.007,326569308


#### Filter for given series_id, year, period

In [28]:
df4 = df3.loc[(df3['series_id'] == 'PRS30006032') & (df3['year'] == 2018) & (df3['period'] == 'Q01')]

In [29]:
df4 = df4.reset_index(drop=True)

#### Final Output

In [30]:
df4

,series_id,year,period,value,Population
0,PRS30006032,2018,Q01,-0.2,322903030
